In [ ]:
import a1chemy.data_source as data_source
import a1chemy.indicators as indicators
import a1chemy.plot.plotly_tools as plotly_tools
import pymongo
import operator
import pandas as pd
from tqdm.notebook import tqdm
import base64
import ipywidgets as widgets
import pandas as pd
import numpy as np
from operator import truediv 

mongo_client = pymongo.MongoClient("mongodb://localhost:27017/", username='a1chemy', password='1B2C9046-E3CC-447F-9961-E125759BA44F')
mongo_ticks_client = data_source.MongoTicks(mongo_client)
mongo_tags_client = data_source.MongoTags(mongo_client)
mongo_fund_client = data_source.MongoFund(mongo_client)
mongo_fund_client = data_source.MongoFund(mongo_client)

def return_rate(t, day1, day2):
    length = t.raw_data.shape[0]
    if -length >= day1 or -length >= day2:
        return -100
    return "{:.3f}".format(100*(t.close().iloc[day1] - t.close().iloc[day2])/t.close().iloc[day2])


def relative(n, d, last_count):
    n_data = n.raw_data.iloc[-last_count:None]
    n_last = n_data['close'].tolist()
    d_last = d.raw_data.iloc[-last_count:None]['close'].tolist()
    relative_data = list(map(truediv, n_last, d_last))
    df = pd.DataFrame({'time': n_data['time'].tolist(), 
                       'relative': relative_data})
    return plotly_tools.line_thumbnail(data=df, x_axis='time', y_axis='relative')

def row_data(exchange, symbol, relative_base, layouts):
    #print('{} {}'.format(exchange, symbol))
    ticks = mongo_ticks_client.find_one(exchange=exchange, symbol=symbol)
    ticks.convert_tz(tz='Asia/Shanghai')
    fund = mongo_fund_client.find_one(exchange=exchange, symbol=symbol)
    
    count=20
    month_thumbnail = plotly_tools.ticks_thumbnail(ticks=ticks, start=-count)
    shares = plotly_tools.fund_amount_thumbnail(fund=fund, start=-count)
    relative_image_widget = None
    if relative_base.symbol == ticks.symbol:
        relative_image_widget = widgets.Label(value='--'.format(ticks.close().iloc[-1]), layout=layouts[6])
    else:
        relative_image = relative(ticks, relative_base, count)
        relative_image_widget = widgets.Image(value=relative_image.data if relative_image else None, layout=layouts[6])
    return widgets.HBox([widgets.Label(value=ticks.name, layout=layouts[0]), 
                     widgets.Label(value=ticks.symbol, layout=layouts[1]), 
                     widgets.Label(value='{:.3f}'.format(ticks.close().iloc[-1]), layout=layouts[2]), 
                     widgets.Label(value=return_rate(ticks, -1, -6), layout=layouts[3]), 
                     widgets.Image(value=month_thumbnail.data, layout=layouts[4]),
                     widgets.Image(value=shares.data, layout=layouts[5]),
                     relative_image_widget])

def show_rows(symbol_list, relative_base, layouts):
    rows_list = [row_data(d[0], d[1], relative_base, layouts) for d in symbol_list]
    return widgets.VBox(rows_list)

def show_title(data, layouts):
    return widgets.HBox([widgets.Label(value=data[i], layout=layouts[i]) for i in range(len(data))])

def market_data_box(title, market_data, relative_base, title_layouts, market_layouts):
    title_box = show_title([title, 'Price', '5D%', '1M', 'Shr', 'RM'], title_layouts)
    rows_box = show_rows(market_data, relative_base, market_layouts)
    
    return widgets.VBox([title_box, rows_box], 
             layout=widgets.Layout(display='flex', border='solid 1px black', width='660px'))

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

row_layouts = [
    widgets.Layout(width='80px'),
    widgets.Layout(width='90px', display='flex', justify_content="center"),
    widgets.Layout(width='50px', display='flex', justify_content="center"),
    widgets.Layout(width='50px', display='flex', justify_content="center", color=''),
    widgets.Layout(width='120px', display='flex', justify_content="center"),
    widgets.Layout(width='120px', display='flex', justify_content="center"),
    widgets.Layout(width='120px', display='flex', justify_content="center")
]

title_layouts = [
    widgets.Layout(width='170px', color='grey'),
    widgets.Layout(width='50px', display='flex', justify_content="center"),
    widgets.Layout(width='50px', display='flex', justify_content="center"),
    widgets.Layout(width='120px', display='flex', justify_content="center"),
    widgets.Layout(width='120px', display='flex', justify_content="center"),
    widgets.Layout(width='120px', display='flex', justify_content="center")
]

board_market_list = [
    ('SH', 'SH515800'),
    ('SH', 'SH510050'),
    ('SH', 'SH510300'),
    ('SH', 'SH510500'),
    ('SZ', 'SZ159915'),
    ('SZ', 'SZ159949'),
    ('SH', 'SH588090'),
    ('SH', 'SH588000'),
    ('SH', 'SH518880'),
    ('SH', 'SH511010'),
    ('SH', 'SH511260')
]
consumer_list = [
    ('SZ', 'SZ159928'),
    ('SH', 'SH515650'),
    ('SZ', 'SZ159996'),
    ('SH', 'SH512690'),
    ('SH', 'SH515030')
]

manufacture_list = [
    ('SH', 'SH515750'),
    #('SH', 'SH515790'),
    ('SZ', 'SZ159997'),
    ('SZ', 'SZ159995'),
    ('SH', 'SH512760'),
    ('SH', 'SH512480'),
]

healthcare_list = [
    ('SZ', 'SZ159938'),
    ('SZ', 'SZ159992'),
    ('SH', 'SH512170'),
    ('SH', 'SH512290')
]

financial_list = [
    ('SH', 'SH510230'),
    ('SH', 'SH512800'),
    ('SH', 'SH512070'),
    ('SH', 'SH512880'),
    ('SH', 'SH512000'),
]

etc_list = [
    ('SH', 'SH512400'),
    ('SH', 'SH512660')
]

relative_base = mongo_ticks_client.find_one(exchange='SH', symbol='SH515800')
relative_base.convert_tz(tz='Asia/Shanghai')

blank_box = widgets.Label(value = '  ', layout=widgets.Layout(width='10px'))

today_date = relative_base.index().iloc[-1].strftime('%Y-%m-%d')
today_date_box = widgets.Label(value='日期:' + today_date, layout=widgets.Layout(display='flex', border='solid 1px black'))
sheet_box = widgets.Label(value = 'Price=收盘价   5D%=五日价格变化幅度   1M=20个交易日价格波动   Shr=ETF份额变化   RM=相对800ETF的变化幅度')


data_box = widgets.HBox([
    widgets.VBox([
        market_data_box('市场概况', board_market_list, relative_base, title_layouts, row_layouts),
        today_date_box,
        sheet_box
    ]),
    blank_box,
    widgets.VBox([
        market_data_box('消费', consumer_list, relative_base, title_layouts, row_layouts),
        market_data_box('金融', financial_list, relative_base, title_layouts, row_layouts),
        market_data_box('医药/医疗', healthcare_list, relative_base, title_layouts, row_layouts),
        market_data_box('制造业', manufacture_list, relative_base, title_layouts, row_layouts),
        market_data_box('Etc', etc_list, relative_base, title_layouts, row_layouts)
    ])
])

print('{}市场概况'.format(today_date))
widgets.VBox([sheet_box, data_box])